# Players dataframes

En este script nos dedicaremos a crear una base de datos limpia segmentada por hitters y fielders. Se divide en las siguientes secciones:

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Construcción de variables para estimaciones.**
- **Unión de las bases de datos para nuevas bases transversales.**
- **Segmentación de bases datos de acuerdo a la agencia libre.**
- **Realización del panel data de acuerdo a las especificaciones.**
- **Generación de las variables para el modelo dinámico.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
import statsmodels.api as sm
from matplotlib.colors import ListedColormap
from termcolor import colored
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')

In [3]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN/ETL_Scripts/Hand_Made_Scripts/First_Year_Contract


In [4]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Visualización de las bases de datos

### Equipos por estado

In [5]:
states = 'Data/Teams/team_states.csv'
df_states = pd.read_csv(states)

In [6]:
df_states.head()

,Estado,Cantidad de equipos
0,Alabama,0
1,Alaska,0
2,Arizona,1
3,Arkansas,0
4,California,5


### Acrónimos

Nos servirá como llave intermedia para unificar las bases de datos de los equipos

In [7]:
acronym = 'Data/Teams/team_acronym.csv'
df_acronym = pd.read_csv(acronym)

In [8]:
df_acronym.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


Unamos esta dataframe con el de los equipos por estado

In [9]:
acronym_state = pd.merge(df_states, df_acronym, on = 'Estado')

In [10]:
acronym_state.head()

,Estado,Cantidad de equipos,Equipo,Acronimo
0,Arizona,1,Arizona Diamondbacks,ARI
1,California,5,Los Angeles Angels,LAA
2,California,5,Los Angeles Dodgers,LAD
3,California,5,Oakland Athletics,OAK
4,California,5,San Diego Padres,SD


En este caso, el nombre de las variables es claro

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [11]:
# Auxiliares:
free_agents = 'Data/Free_Agents/free_agents_'
hitting = 'Data/Statistics/Cumulative/Hitting/hitting_'
pitching = 'Data/Statistics/Cumulative/Pitching/pitching_'
salary = 'Data/Salary/salary_'
teams = 'ETL_Data/Transversal/Teams/free_agents_team_'
csv = '.csv'
period = 12
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period
df_panel_hitters = [None]*period
df_panel_pitchers = [None]*period

Leamos todos los archivos y creemos las copias

In [12]:
for year in range(0,period):    
    df_free_agents[year] = pd.read_csv(free_agents + str(2011 + year) + csv)
    df_hitting[year] = pd.read_csv(hitting + str(2011 + year) + csv)
    df_pitching[year] = pd.read_csv(pitching + str(2011 + year) + csv)
    df_salary[year] = pd.read_csv(salary + str(2011 + year) + csv)
    df_teams[year] = pd.read_csv(teams + str(2011 + year) + csv)
    
    df_free_agents_copy[year] = df_free_agents[year].copy()
    df_hitting_copy[year] = df_hitting[year].copy()
    df_pitching_copy[year] = df_pitching[year].copy()
    df_salary_copy[year] = df_salary[year].copy()
    df_teams_copy[year] = pd.read_csv(teams + str(2011 + year) + csv)

Tratemos las bases de datos por separado. Sin embargo, a todas les quitaremos la columna de rango y *Cash2023*.

Como no queremos que se repita la columna del año de la temporada de la base de datos, borremos la columna de *Year* de la base  de datos de los agentes libres. Como los años del contrato aparecen en la base de datos sobre los salarios, se prefiere conservar dicha columna en la base de datos de salarios puesto que esta base de datos es más general que la de los agentes libres, razón por la que se borrará de esta última base de datos. 

El equipo al que se cambia el agente libre está señalado por la columna del equipo en la base de datos de salarios y la estadísticas deportivas por lo que se borrará *Team From To* de la base de datos de los agentes libres. 

Como nos importan los salarios para este analisis, quitaremos la columna de los equipos en las bases de datos sobre las estadísticas deportivas de todos los jugadores, así como la posición que ocupan.

In [13]:
for year in range(0,period):
    # Drop columns:
    if any(name in df_free_agents_copy[year].columns for name in ['Rank','Pos','Year','Team From To']):
        df_free_agents_copy[year].drop('Rank', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Year', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Pos', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Team From To', axis = 1, inplace = True)
    if 'Rank' in df_salary_copy[year].columns:
        df_salary_copy[year].drop('Rank', axis = 1, inplace = True)
    if any(name in df_hitting_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_hitting_copy[year].drop('Rank', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Team', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Pos', axis = 1, inplace = True)
    if any(name in df_pitching_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_pitching_copy[year].drop('Rank', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Team', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Pos', axis = 1, inplace = True)

Debido a que aparecen columnas que inician con el  nombre *Unnamed*, tendremos que borrarlas con algún método general, el cual se muestra a continuación:

In [14]:
for year in range(0,period):
    # Base de datos de agentes libres:
    df_free_agents_copy[year].drop(df_free_agents_copy[year].columns[df_free_agents_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los salarios:
    df_salary_copy[year].drop(df_salary_copy[year].columns[df_salary_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los hitters:
    df_hitting_copy[year].drop(df_hitting_copy[year].columns[df_hitting_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los pitchers:
    df_pitching_copy[year].drop(df_pitching_copy[year].columns[df_pitching_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Verifiquemos que ya no se encuentran dichas columnas molestas

In [15]:
df_free_agents_copy[9].columns

Index(['Player', 'Status', 'Team From', 'YRS', 'Value', 'AAV'], dtype='object')

In [16]:
df_salary_copy[11].columns

Index(['Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height'],
      dtype='object')

In [17]:
df_hitting_copy[2].columns

Index(['Player', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'TVS'],
      dtype='object')

In [18]:
df_pitching_copy[5].columns

Index(['Player', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV',
       'WHIP', 'ERA', 'WAR', 'TVS'],
      dtype='object')

#### Agentes libres

No se conservará el equipo al que es contratado el agente libre puesto que esta información también la contiene la base de datos que facilita más el tratamiento _ETL_.

In [19]:
for year in range(0,period):
    df_free_agents_copy[year] = df_free_agents_copy[year].rename(columns = {'Player':'Jugador',
                                'Status':'Status_agente_libre', 'Team From':'Equipo_anterior',
                                'Value':'Valor_contrato', 'AAV':'Valor_promedio_contrato',
                                'YRS':'Anios_de_contrato'})
    
    free_agents_aux_1 = df_free_agents_copy[year]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[year]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[year]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[year]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[year]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_contrato'])
    df_free_agents_copy[year]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_promedio_contrato'])

Observemos las dimensiones de las bases de datos como referencia

In [20]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(213, 6)
(208, 6)
(221, 6)
(241, 6)
(100, 6)
(98, 6)
(105, 6)
(118, 6)
(141, 6)
(137, 6)


También el tipo de datos que contiene cada columna

In [21]:
df_free_agents_copy[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Jugador                  100 non-null    object
 1   Status_agente_libre      100 non-null    object
 2   Equipo_anterior          100 non-null    object
 3   Anios_de_contrato        100 non-null    int64 
 4   Valor_contrato           100 non-null    int64 
 5   Valor_promedio_contrato  100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


#### Salarios

Como los salarios irán con las bases de datos de los _hitters_ y _pitchers_ es que se hará su proceso _ETL_ antes.

In [22]:
for year in range(0,period):
    # Cambio de nombres
    df_salary_copy[year] = df_salary_copy[year].rename(columns = {'Player':'Jugador',
                            'BaseSalary':'Sueldo_base', 'SigningBonus':'Bono_por_firma',
                            'Payroll Salary':'Sueldo_regular', 'Adj Salary':'Sueldo_ajustado',
                            'CONT YR':'Anios_de_contrato', 'CONT VALUE':'Valor_del_contrato',
                            'Earnings':'Ganancias', 'FA Year':'Anio_de_agente_libre',
                            'Sign Age':'Edad_al_firmar', 'Age':'Edad', 'Weight':'Peso',
                            'Height':'Altura', 'Year':'Anio', 'Pos':'Posicion',
                            'Salary%':'Sueldo_porcentual', 'Cash':'Pago_efectivo',
                            'AAV':'Valor_contrato_promedio', 'Team':'Acronimo'})
    
    # Tranformando al tipo de dato apropiado
    salary_aux_1 = df_salary_copy[year]['Sueldo_base'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[year]['Sueldo_base'] = salary_aux_2
    df_salary_copy[year]['Sueldo_base'] = pd.to_numeric(df_salary_copy[year]['Sueldo_base'])
    
    salary_aux_3 = df_salary_copy[year]['Sueldo_regular'].str.replace("$","")
    salary_aux_4 = salary_aux_3.str.replace(",","")
    df_salary_copy[year]['Sueldo_regular'] = salary_aux_4
    df_salary_copy[year]['Sueldo_regular'] = pd.to_numeric(df_salary_copy[year]['Sueldo_regular'])
    
    salary_aux_5 = df_salary_copy[year]['Sueldo_ajustado'].str.replace("$","")
    salary_aux_6 = salary_aux_5.str.replace(",","")
    df_salary_copy[year]['Sueldo_ajustado'] = salary_aux_6
    df_salary_copy[year]['Sueldo_ajustado'] = pd.to_numeric(df_salary_copy[year]['Sueldo_ajustado'])
    
    salary_aux_7 = df_salary_copy[year]['Valor_del_contrato'].str.replace("$","")
    salary_aux_8 = salary_aux_7.str.replace(",","")
    df_salary_copy[year]['Valor_del_contrato'] = salary_aux_8
    df_salary_copy[year]['Valor_del_contrato'] = pd.to_numeric(df_salary_copy[year]['Valor_del_contrato'])
    
    salary_aux_9 = df_salary_copy[year]['Bono_por_firma'].str.replace("$","")
    salary_aux_10 = salary_aux_9.str.replace(",","")
    df_salary_copy[year]['Bono_por_firma'] = salary_aux_10
    df_salary_copy[year]['Bono_por_firma'] = pd.to_numeric(df_salary_copy[year]['Bono_por_firma'])
    
    salary_aux_11 = df_salary_copy[year]['Ganancias'].str.replace("$","")
    salary_aux_12 = salary_aux_11.str.replace(",","")
    df_salary_copy[year]['Ganancias'] = salary_aux_12
    df_salary_copy[year]['Ganancias'] = pd.to_numeric(df_salary_copy[year]['Ganancias'])
    
    salary_aux_13 = df_salary_copy[year]['Pago_efectivo'].str.replace("$","")
    salary_aux_14 = salary_aux_13.str.replace(",","")
    df_salary_copy[year]['Pago_efectivo'] = salary_aux_14
    df_salary_copy[year]['Pago_efectivo'] = pd.to_numeric(df_salary_copy[year]['Pago_efectivo'])
    
    salary_aux_15 = df_salary_copy[year]['Valor_contrato_promedio'].str.replace("$","")
    salary_aux_16 = salary_aux_15.str.replace(",","")
    df_salary_copy[year]['Valor_contrato_promedio'] = salary_aux_16
    df_salary_copy[year]['Valor_contrato_promedio'] = pd.to_numeric(df_salary_copy[year]['Valor_contrato_promedio'])
    
    salary_aux_17 = df_salary_copy[year]['Altura'].str.replace("\"","")
    salary_aux_18 = salary_aux_17.str.replace("'","")
    df_salary_copy[year]['Altura'] = salary_aux_18
    df_salary_copy[year]['Altura'] = pd.to_numeric(df_salary_copy[year]['Altura'])/10
    # SUstitullamos los xeros
    height_mean = df_salary_copy[year]['Altura'].mean(skipna=True)
    df_salary_copy[year]['Altura'] = df_salary_copy[year].Altura.mask(df_salary_copy[year].Altura == 0, height_mean)
    
    df_salary_copy[year]['Anio_de_agente_libre'] = pd.to_numeric(df_salary_copy[year]['Anio_de_agente_libre'])
    df_salary_copy[year]['Anios_de_contrato'] = pd.to_numeric(df_salary_copy[year]['Anios_de_contrato'])
    df_salary_copy[year]['Edad'] = pd.to_numeric(df_salary_copy[year]['Edad'])

Por algunas particularidades de la base de datos, las columna que contiene la edad al firmar se tratará por separado aprovechando que la mayoría de los datos incorrectos tienen una longitud mayor a dos.

In [23]:
for year in range (0,period):
    df_salary_copy[year]['Edad_al_firmar'] = df_salary_copy[year]['Edad_al_firmar'].map(str)

    for edad in range(0,df_salary_copy[year].shape[0]):
        # String es mayor que 0:
        if len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) == 2:
            df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])
            
        # String es menor o igual que 0:
        elif len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != 2:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)    
    
        # Entero  menor a 0:
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)
         
    # Transformemos los datos a enteros
    df_salary_copy[year]['Edad_al_firmar'] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'])

Podemos verificar si se limpiaron adecuadamente las celdas de la columna de edades al firmar. Esto, al filtrar los datos que sean distintos a enteros y al observar si se pudo transformar toda la columna al tipo entero.

In [24]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != np.int64:
            print(type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]))

In [25]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            print(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])

In [26]:
#for year in range(0,period):
#    print(type(df_salary_copy[year][['Edad_al_firmar']].info()))

In [27]:
#for year in range(0,period):
#    print(year)
#    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
#        print(str(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) + ' ' + str(edad))

Por otro lado, falta corregir las entradas de las columnas de las edades que tengan valores menores a cero. Esto se hará de acuerdo al resto de columnas

In [28]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(df_salary_copy[year]['Edad'].iloc[edad])

0
-9
1
-9
2
-7


In [29]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year].shape[0]):
        # Condición para imputar:
        if df_salary_copy[year]['Edad'].iloc[edad] <= 0:
            # Si no se indica si tendrá año de agencia libre:
            if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                        ag_year = year + 2011 + 1
            # Si tendrá año de agencia libre
            else:
                ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
            # Get first year of contract
            ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
            # Años desde el el año inicial
            dif_years = year + 2011 - ini_year
            # Edad en la temporada:
            seasson_age = df_salary_copy[year]['Edad_al_firmar'].iloc[edad] + dif_years
            # Asignación
            df_salary_copy[year]['Edad'].iloc[edad] = seasson_age

Comprobemos que no hay ninguna edad negativa

In [30]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(str(df_salary_copy[year]['Edad'].iloc[edad]) + ' ' + str(edad))

Con la imputación de datos ya se puede crear la columna que contiene la antiguedad del agente libre bajo el contrato

In [31]:
for year in range(0,period):
    df_salary_copy[year]['Antiguedad'] = df_salary_copy[year]['Edad'] - df_salary_copy[year]['Edad_al_firmar']

Por último, convirtamos la columna del año a string para que se entienda como una categoría y no una variable numérica

In [32]:
for year in range(0,period):
    df_salary_copy[year]['Anio'] = df_salary_copy[year]['Anio'].map(str)

In [33]:
df_salary_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1045 non-null   object 
 1   Anio                     1045 non-null   object 
 2   Posicion                 1045 non-null   object 
 3   Acronimo                 1045 non-null   object 
 4   Sueldo_base              1045 non-null   int64  
 5   Bono_por_firma           1045 non-null   int64  
 6   Sueldo_regular           1045 non-null   int64  
 7   Sueldo_ajustado          1045 non-null   int64  
 8   Sueldo_porcentual        1045 non-null   float64
 9   Pago_efectivo            1045 non-null   int64  
 10  Valor_contrato_promedio  1045 non-null   int64  
 11  Anios_de_contrato        1045 non-null   int64  
 12  Valor_del_contrato       1045 non-null   int64  
 13  Ganancias                1045 non-null   int64  
 14  Anio_de_agente_libre    

#### Hitters

In [34]:
for year in range(0,period):
    # Cambio de nombres
    df_hitting_copy[year] = df_hitting_copy[year].rename(columns = {'Player':'Jugador',
                            'GP':'Juegos', 'GP%':'Porcentaje_juegos',
                            'AB':'At_bats', 'H':'Bateos', 'GS':'Juegos_iniciados',
                            'GS%':'Porcentaje_juegos_iniciados', 'RBI':'Runs_batted_in',
                            'HR':'Home_runs', 'AVG':'Bateos_promedio',
                            '2B':'Dobles', '3B':'Triples', 'OPS':'Porcentaje_On_base_plus_slugging',
                            'SLG':'Porcentaje_slugging', 'OBP':'Porcentaje_on_base'})

In [35]:
for year in range(0,period):
    print(df_hitting_copy[year].shape)

(521, 17)
(669, 17)
(734, 17)
(778, 17)
(809, 17)
(834, 17)
(810, 17)
(825, 17)
(850, 17)
(451, 17)
(753, 17)
(496, 17)


In [36]:
df_hitting_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Jugador                           834 non-null    object 
 1   Juegos                            834 non-null    int64  
 2   Porcentaje_juegos                 834 non-null    float64
 3   Juegos_iniciados                  834 non-null    int64  
 4   Porcentaje_juegos_iniciados       834 non-null    float64
 5   At_bats                           834 non-null    int64  
 6   Bateos                            834 non-null    int64  
 7   Dobles                            834 non-null    int64  
 8   Triples                           834 non-null    int64  
 9   Home_runs                         834 non-null    int64  
 10  Runs_batted_in                    834 non-null    int64  
 11  Bateos_promedio                   834 non-null    float64
 12  Porcenta

In [37]:
df_hitting_copy[5].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At_bats', 'Bateos', 'Dobles', 'Triples',
       'Home_runs', 'Runs_batted_in', 'Bateos_promedio', 'Porcentaje_on_base',
       'Porcentaje_slugging', 'Porcentaje_On_base_plus_slugging', 'WAR',
       'TVS'],
      dtype='object')

#### Pitchers

In [38]:
for year in range(0,period):
    # Cambio de nombres
    df_pitching_copy[year] = df_pitching_copy[year].rename(columns = {'Player':'Jugador',
                             'GP':'Juegos', 'GS':'Juegos_iniciados', 'IP':'Inning_pitched',
                             'H':'Bateos', 'R':'Carreras', 'ER':'Carreras_ganadas',
                             'BB':'Walks', 'SO':'Strike_outs', 'W':'Wins', 'L':'Losses',
                             'SV':'Saves'})

In [39]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 16)
(427, 16)
(485, 16)
(537, 16)
(586, 16)
(592, 16)
(581, 16)
(644, 16)
(668, 16)
(542, 16)
(611, 16)
(598, 16)


In [40]:
df_pitching_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           592 non-null    object 
 1   Juegos            592 non-null    int64  
 2   Juegos_iniciados  592 non-null    int64  
 3   Inning_pitched    592 non-null    float64
 4   Bateos            592 non-null    int64  
 5   Carreras          592 non-null    int64  
 6   Carreras_ganadas  592 non-null    int64  
 7   Walks             592 non-null    int64  
 8   Strike_outs       592 non-null    int64  
 9   Wins              592 non-null    int64  
 10  Losses            592 non-null    int64  
 11  Saves             592 non-null    int64  
 12  WHIP              592 non-null    float64
 13  ERA               592 non-null    float64
 14  WAR               587 non-null    float64
 15  TVS               592 non-null    float64
dtypes: float64(5), int64(10), object(1)
memory u

## Agregación de variables sugeridas por artículos

Las primeras variables que agregaremos son el cuadrado de todas las estadísticas deportivas, así como las siguientes variables:

- DOMINANCE = $Strike-outs/(Inning \; Pitched)$
- CONTROL = $Walks/(Inning \; Pitched)$
- COMMAND = $Strike-outs/Walks$

In [41]:
for year in range(0,period):
    df_pitching_copy[year]['Dominio'] = df_pitching_copy[year]['Strike_outs']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Control'] = df_pitching_copy[year]['Walks']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Comando'] = df_pitching_copy[year]['Strike_outs']/df_pitching_copy[year]['Walks']

In [42]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 19)
(427, 19)
(485, 19)
(537, 19)
(586, 19)
(592, 19)
(581, 19)
(644, 19)
(668, 19)
(542, 19)
(611, 19)
(598, 19)


In [43]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           485 non-null    object 
 1   Juegos            485 non-null    int64  
 2   Juegos_iniciados  485 non-null    int64  
 3   Inning_pitched    485 non-null    float64
 4   Bateos            485 non-null    int64  
 5   Carreras          485 non-null    int64  
 6   Carreras_ganadas  485 non-null    int64  
 7   Walks             485 non-null    int64  
 8   Strike_outs       485 non-null    int64  
 9   Wins              485 non-null    int64  
 10  Losses            485 non-null    int64  
 11  Saves             485 non-null    int64  
 12  WHIP              485 non-null    float64
 13  ERA               485 non-null    float64
 14  WAR               435 non-null    float64
 15  TVS               485 non-null    float64
 16  Dominio           484 non-null    float64
 1

Podemos verificar qué entradas poseen valores infinitos en la base de datos

In [44]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Siguiendo la sugerencia de algunos artículos, obtengamos el logaritmo de los salarios

In [45]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_base'] = np.log(df_salary_copy[year]['Sueldo_base'])
    df_salary_copy[year]['ln_Sueldo_ajustado'] = np.log(df_salary_copy[year]['Sueldo_ajustado'])
    df_salary_copy[year]['ln_Sueldo_regular'] = np.log(df_salary_copy[year]['Sueldo_regular'])

In [46]:
df_salary_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1013 non-null   object 
 1   Anio                     1013 non-null   object 
 2   Posicion                 1013 non-null   object 
 3   Acronimo                 1013 non-null   object 
 4   Sueldo_base              1013 non-null   int64  
 5   Bono_por_firma           1013 non-null   int64  
 6   Sueldo_regular           1013 non-null   int64  
 7   Sueldo_ajustado          1013 non-null   int64  
 8   Sueldo_porcentual        1013 non-null   float64
 9   Pago_efectivo            1013 non-null   int64  
 10  Valor_contrato_promedio  1013 non-null   int64  
 11  Anios_de_contrato        1013 non-null   int64  
 12  Valor_del_contrato       1013 non-null   int64  
 13  Ganancias                1013 non-null   int64  
 14  Anio_de_agente_libre    

Debido a que hay columnas con datos tipo _Nan_ o _NULL_, optaremos por imputarlos.

Mientras que los valores infinitos generados por las nuevas variables se sustituirán dependediendo del caso:

- 0/0: 0
- num/0: Máximo de la columna correspondientefijarán

In [47]:
for year in range(0,period):
    # Salaries
    mean_hgt = df_salary_copy[year].loc[df_salary_copy[year]['Altura'] > 4.9].Altura.mean()
    mean_wgh = df_salary_copy[year].loc[df_salary_copy[year]['Peso'] > 0].Peso.mean()
    df_salary_copy[year]['Altura'].fillna(value = mean_hgt, inplace = True)
    df_salary_copy[year]['Altura'].mask(df_salary_copy[year]['Altura'] <= 4.9, mean_hgt, inplace = True)
    df_salary_copy[year]['Peso'].fillna(value = mean_wgh, inplace = True)
    df_salary_copy[year]['Peso'].mask(df_salary_copy[year]['Peso'] <= 0, mean_wgh, inplace = True)
    
    # Pitchers
    mean_war = df_pitching_copy[year].loc[df_pitching_copy[year]['WAR'] > 0].WAR.mean()
    mean_dom = df_pitching_copy[year].loc[df_pitching_copy[year]['Dominio'] > 0].Dominio.mean()
    mean_con = df_pitching_copy[year].loc[df_pitching_copy[year]['Control'] > 0].Control.mean()
    mean_com = df_pitching_copy[year].loc[df_pitching_copy[year]['Comando'] > 0].Comando.mean()
    df_pitching_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_pitching_copy[year]['WAR'].mask(df_pitching_copy[year]['WAR'] <= 0, mean_war, inplace = True)
    df_pitching_copy[year]['Dominio'].fillna(value = mean_dom, inplace = True)
    df_pitching_copy[year]['Dominio'].mask(df_pitching_copy[year]['Dominio'] <= 0, mean_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = mean_con, inplace = True)
    df_pitching_copy[year]['Control'].mask(df_pitching_copy[year]['Control'] <= 0, mean_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = mean_com, inplace = True)
    df_pitching_copy[year]['Comando'].mask(df_pitching_copy[year]['Comando'] <= 0, mean_com, inplace = True)
    
    # Hitters
    mean_war = df_hitting_copy[year].loc[df_hitting_copy[year]['WAR'] > 0].WAR.mean()
    df_hitting_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_hitting_copy[year]['WAR'].mask(df_hitting_copy[year]['WAR'] <= 0, mean_war, inplace = True)

In [48]:
for year in range(0,period):   
    # Condiciones
    con_dom_1 = df_pitching_copy[year]['Strike_outs'] == 0
    con_con_1 = df_pitching_copy[year]['Walks'] == 0
    con_com_1 = df_pitching_copy[year]['Strike_outs'] == 0
                 
    # Imputación caso 0/0
    df_pitching_copy[year].loc[con_dom_1, "Dominio"] = 0
    df_pitching_copy[year].loc[con_con_1, "Control"] = 0
    df_pitching_copy[year].loc[con_com_1, "Comando"] = 0

In [49]:
for year in range(0,period):   
    # Máximos
    max_dom = df_pitching_copy[year]['Strike_outs'].max()/9
    max_con = df_pitching_copy[year]['Walks'].max()/9
    max_com = df_pitching_copy[year]['Strike_outs'].max()
    
    # Cambianfdo infinitos a NaNs
    df_pitching_copy[year]["Dominio"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Control"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Comando"].replace([np.inf, -np.inf], np.nan, inplace = True)
    
    # Imputación
    df_pitching_copy[year]['Dominio'].fillna(value = max_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = max_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = max_com, inplace = True)

Verifiquemos que ya no haya problemas con valores infinitos

In [50]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Así mismo, contemos los valores *NaN* que queden presentes

In [51]:
for year in range(0,period):
    print('Año: ' + str(2011 + year))
    print('Hitters:')
    df_hitting_copy[year].isna().sum()
    print('Pitchers:')
    df_pitching_copy[year].isna().sum()
    print('Free agents:')
    df_free_agents_copy[year].isna().sum()
    print('Salaries:')
    df_salary_copy[year].isna().sum()
    print("")

Año: 2011
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2012
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2013
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2014
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2015
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2016
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2017
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2018
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2019
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2020
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2021
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2022
Hitters:
Pitchers:
Free agents:
Salaries:



Ahora, repitamos este proceso para la base de datos de los salarios.

In [52]:
salary_names = ['ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular']

In [53]:
for name in salary_names:
    print(name)
    
    for year in range(0,period):
        print(str(2011 + year))
        for element in range(0,len(df_salary_copy[year][name])):
            if df_salary_copy[year][name].iloc[element] <= 0:
                print(str(element) +  '  ' + str(df_salary_copy[year][name].iloc[element]))
        print("")

ln_Sueldo_ajustado
2011

2012

2013

2014

2015

2016

2017

2018
72  -inf
188  -inf

2019

2020

2021
6  -inf
55  -inf
166  -inf
274  -inf

2022
193  -inf

ln_Sueldo_base
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022

ln_Sueldo_regular
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



Al inspecsionar los errores nos damos cuenta que solo se desconocen los salarios ajustados y los fijaron a $0$. Usaremos el logaritmo base 10 del salario regular para sustituir dicho valor.

In [54]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_ajustado'].mask(df_salary_copy[year]['ln_Sueldo_ajustado'] < 0,
                                                    df_salary_copy[year]['ln_Sueldo_regular'],
                                                    inplace = True)

In [55]:
for year in range(0,period):
    print(str(2011 + year))
    for element in range(0,len(df_salary_copy[year]['ln_Sueldo_ajustado'])):
        if df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element] <= 0:
            print(str(element) +  '  ' + str(df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element]))
    print("")

2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



In [56]:
for year in range(0,period):
    print("Ajustado: " + str(df_salary_copy[year]['ln_Sueldo_ajustado'].mean())
          + 'n'
          + 'Regular: ' + str(df_salary_copy[year]['ln_Sueldo_regular'].mean()))

Ajustado: 13.785784770849762nRegular: 13.888874386883515
Ajustado: 13.81181498978195nRegular: 13.90039782504837
Ajustado: 13.54819591748778nRegular: 13.978975725734408
Ajustado: 13.66411021920866nRegular: 14.080738990191257
Ajustado: 13.49415866539551nRegular: 13.99344595993313
Ajustado: 13.478992231977141nRegular: 14.0261405412466
Ajustado: 13.575363447521962nRegular: 14.079742661574734
Ajustado: 13.562311948859815nRegular: 14.080756222584538
Ajustado: 13.521364643693584nRegular: 14.066559580453122
Ajustado: 13.029458522392627nRegular: 14.216780831796356
Ajustado: 13.70981799580226nRegular: 14.198804602895482
Ajustado: 13.924314685157658nRegular: 14.351734783233747


En efecto, ya no hay valores _NaN_ o _infinitos_.

Con el objetivo de hacer más eficiente la creación de las variables al cuadrado, lo haremos extrayendo el índice de las columnas de interés

In [57]:
df_hitting_copy[0].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At_bats', 'Bateos', 'Dobles', 'Triples',
       'Home_runs', 'Runs_batted_in', 'Bateos_promedio', 'Porcentaje_on_base',
       'Porcentaje_slugging', 'Porcentaje_On_base_plus_slugging', 'WAR',
       'TVS'],
      dtype='object')

In [58]:
df_pitching_copy[1].columns

Index(['Jugador', 'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos',
       'Carreras', 'Carreras_ganadas', 'Walks', 'Strike_outs', 'Wins',
       'Losses', 'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
       'Comando'],
      dtype='object')

In [59]:
def get_col_indices(df, names):
    return df.columns.get_indexer(names)

In [60]:
hitting_names = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At_bats', 'Bateos',
                  'Dobles', 'Triples', 'Home_runs', 'Runs_batted_in', 'Bateos_promedio',
                  'Porcentaje_on_base', 'Porcentaje_slugging', 'TVS',
                  'Porcentaje_On_base_plus_slugging', 'WAR']	
pitching_names = ['Inning_pitched', 'Bateos', 'Carreras',
                  'Carreras_ganadas', 'Walks', 'Strike_outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando']

Con el objetivo de simplificar el código, verifiquemos si todos los índices en cada base de datos son los mismos

In [61]:
print('Hitters:')
for year in range(0,period):
    print(get_col_indices(df_hitting_copy[year], hitting_names))
    
print('Pitchers:')
for year in range(0,period):
    print(get_col_indices(df_pitching_copy[year], pitching_names))

Hitters:
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
Pitchers:
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3

In [62]:
hitting_indexes = list(get_col_indices(df_hitting_copy[0], hitting_names))
pitching_indexes = list(get_col_indices(df_pitching_copy[0], pitching_names))

In [63]:
for year in range(0,period):
    # Hitters:
    for hitter_name in hitting_indexes:
        df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name] + '_2'] = np.power(df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name]], 2)
    # Pitchers:
    for pitcher_name in pitching_indexes:
        df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name] + '_2'] = np.power(df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name]], 2)

Apreciemos el resultado final

In [64]:
df_hitting_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             734 non-null    object 
 1   Juegos                              734 non-null    int64  
 2   Porcentaje_juegos                   734 non-null    float64
 3   Juegos_iniciados                    734 non-null    int64  
 4   Porcentaje_juegos_iniciados         734 non-null    float64
 5   At_bats                             734 non-null    int64  
 6   Bateos                              734 non-null    int64  
 7   Dobles                              734 non-null    int64  
 8   Triples                             734 non-null    int64  
 9   Home_runs                           734 non-null    int64  
 10  Runs_batted_in                      734 non-null    int64  
 11  Bateos_promedio                     734 non-n

In [65]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             485 non-null    object 
 1   Juegos              485 non-null    int64  
 2   Juegos_iniciados    485 non-null    int64  
 3   Inning_pitched      485 non-null    float64
 4   Bateos              485 non-null    int64  
 5   Carreras            485 non-null    int64  
 6   Carreras_ganadas    485 non-null    int64  
 7   Walks               485 non-null    int64  
 8   Strike_outs         485 non-null    int64  
 9   Wins                485 non-null    int64  
 10  Losses              485 non-null    int64  
 11  Saves               485 non-null    int64  
 12  WHIP                485 non-null    float64
 13  ERA                 485 non-null    float64
 14  WAR                 485 non-null    float64
 15  TVS                 485 non-null    float64
 16  Dominio 

## Unión de las bases de datos
### Datos agregados por equipo

Solo resta añadir los datos relevantes al equipo al que pertenece cada jugador considerando la base de datos de la cantidad de equipos por estado

In [66]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
dtypes: float64(1), int64(7), object(2)
memory usage: 2.3+ KB


In [67]:
acronym_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Estado               30 non-null     object
 1   Cantidad de equipos  30 non-null     int64 
 2   Equipo               30 non-null     object
 3   Acronimo             30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [68]:
for year in range(0,period):
    df_teams_copy[year] = pd.merge(df_teams_copy[year], acronym_state, on = ['Equipo','Acronimo'])

In [69]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
 10  Estado                   28 non-null     object 
 11  Cantidad de equipos      28 non-null     int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.8+ KB


In [70]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1036 non-null   object 
 1   Anio                     1036 non-null   object 
 2   Posicion                 1036 non-null   object 
 3   Acronimo                 1036 non-null   object 
 4   Sueldo_base              1036 non-null   int64  
 5   Bono_por_firma           1036 non-null   int64  
 6   Sueldo_regular           1036 non-null   int64  
 7   Sueldo_ajustado          1036 non-null   int64  
 8   Sueldo_porcentual        1036 non-null   float64
 9   Pago_efectivo            1036 non-null   int64  
 10  Valor_contrato_promedio  1036 non-null   int64  
 11  Anios_de_contrato        1036 non-null   int64  
 12  Valor_del_contrato       1036 non-null   int64  
 13  Ganancias                1036 non-null   int64  
 14  Anio_de_agente_libre    

Ahora, unamos las bases de datos sobre los equipos a las bases de datos de los salarios

In [71]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(df_teams_copy[year], df_salary_copy[year], on = 'Acronimo')

In [72]:
df_salary_copy[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   40 non-null     object 
 1   Cantidad_agentes_libres  40 non-null     int64  
 2   Valor_contrato_total     40 non-null     int64  
 3   Acronimo                 40 non-null     object 
 4   Victorias                40 non-null     int64  
 5   Juegos totales           40 non-null     int64  
 6   Playoffs                 40 non-null     int64  
 7   Pennants won             40 non-null     int64  
 8   WS ganadas               40 non-null     int64  
 9   Promedio_victorias       40 non-null     float64
 10  Estado                   40 non-null     object 
 11  Cantidad de equipos      40 non-null     int64  
 12  Jugador                  40 non-null     object 
 13  Anio                     40 non-null     object 
 14  Posicion                 40 

Debido a que la mayoría de los jugadores juega tanto en la ofensiva como la defensiva es que tenemos que borrar los duplicados de la columna de la posición.

In [73]:
for year in range(0,period):
    df_hitting_copy[year] = pd.merge(df_hitting_copy[year], df_salary_copy[year], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitching_copy[year], df_salary_copy[year], on = 'Jugador')

In [74]:
for year in range(0,period):
    df_pitching_copy[year]['Porcentaje_juegos'] = df_pitching_copy[year]['Juegos']/df_pitching_copy[year]['Juegos totales']

In [75]:
df_hitting_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 667 entries, 0 to 666
Data columns (total 64 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             667 non-null    object 
 1   Juegos                              667 non-null    int64  
 2   Porcentaje_juegos                   667 non-null    float64
 3   Juegos_iniciados                    667 non-null    int64  
 4   Porcentaje_juegos_iniciados         667 non-null    float64
 5   At_bats                             667 non-null    int64  
 6   Bateos                              667 non-null    int64  
 7   Dobles                              667 non-null    int64  
 8   Triples                             667 non-null    int64  
 9   Home_runs                           667 non-null    int64  
 10  Runs_batted_in                      667 non-null    int64  
 11  Bateos_promedio                     667 non-n

In [76]:
df_pitching_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 0 to 496
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  497 non-null    object 
 1   Juegos                   497 non-null    int64  
 2   Juegos_iniciados         497 non-null    int64  
 3   Inning_pitched           497 non-null    float64
 4   Bateos                   497 non-null    int64  
 5   Carreras                 497 non-null    int64  
 6   Carreras_ganadas         497 non-null    int64  
 7   Walks                    497 non-null    int64  
 8   Strike_outs              497 non-null    int64  
 9   Wins                     497 non-null    int64  
 10  Losses                   497 non-null    int64  
 11  Saves                    497 non-null    int64  
 12  WHIP                     497 non-null    float64
 13  ERA                      497 non-null    float64
 14  WAR                      4

Para facilitar la observación de las trnasformaciones de manera más eficiente, ordenemos alfabéticamente la base de datos de acuerdo al nombre de las columnas.

In [77]:
for year in range(0,period):
    # Ordenando alfabéticamente
    df_salary_copy[year].sort_index(axis = 1, inplace = True)
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    df_free_agents_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando los índices
    df_salary_copy[year].reset_index(drop = True, inplace = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    df_free_agents_copy[year].reset_index(drop = True, inplace = True)

## Segmentación por Agentes libres

Separaremos los pitchers y hitters en dos grupos:

- Agentes libres.
- No agentes libres.

In [78]:
for year in range(0,period):
    # Filtrando los agentes libres
    df_hitters_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                            df_hitting_copy[year], on = 'Jugador')
    df_pitchers_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                             df_pitching_copy[year], on = 'Jugador')
    # FIltrando los que no son agentes libres
    df_hitters_no_free_agents[year] = df_hitting_copy[year][~df_hitting_copy[year].Jugador.isin(df_hitters_free_agents[year].Jugador)]
    df_pitchers_no_free_agents[year] = df_pitching_copy[year][~df_pitching_copy[year].Jugador.isin(df_pitchers_free_agents[year].Jugador)]
    
    # Reiniciando el índice
    df_hitters_free_agents[year] = df_hitters_free_agents[year].reindex(sorted(df_hitters_free_agents[year].columns), axis=1)
    df_pitchers_free_agents[year] = df_pitchers_free_agents[year].reindex(sorted(df_pitchers_free_agents[year].columns), axis=1)
    df_hitters_no_free_agents[year] = df_hitters_no_free_agents[year].reindex(sorted(df_hitters_no_free_agents[year].columns), axis=1)
    df_pitchers_no_free_agents[year] = df_pitchers_no_free_agents[year].reindex(sorted(df_pitchers_no_free_agents[year].columns), axis=1)    

Veamos los contenidos de las nuevas bases de datos

In [79]:
print("FA - Hitters:")
df_hitters_free_agents[9].info()
print("\n FA - Pitchers:")
df_pitchers_free_agents[9].info()
print("\n No FA - Hitters:")
df_hitters_no_free_agents[9].info()
print("\n No FA - Hitters:")
df_pitchers_no_free_agents[9].info()

FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 0 to 38
Data columns (total 69 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            39 non-null     object 
 1   Altura                              39 non-null     float64
 2   Anio                                39 non-null     object 
 3   Anio_de_agente_libre                39 non-null     int64  
 4   Anios_de_contrato_x                 39 non-null     int64  
 5   Anios_de_contrato_y                 39 non-null     int64  
 6   Antiguedad                          39 non-null     int64  
 7   At_bats                             39 non-null     int64  
 8   At_bats_2                           39 non-null     int64  
 9   Bateos                              39 non-null     int64  
 10  Bateos_2                            39 non-null     int64  
 11  Bateos_promedio                  

In [80]:
print("FA - Hitters:")
for year in range(0,period):
    print(df_hitters_free_agents[year].shape)
print("\n FA - Pitchers:")
for year in range(0,period):
    print(df_pitchers_free_agents[year].shape)

FA - Hitters:
(0, 69)
(64, 69)
(70, 69)
(75, 69)
(69, 69)
(93, 69)
(53, 69)
(51, 69)
(58, 69)
(39, 69)
(63, 69)
(35, 69)

 FA - Pitchers:
(0, 74)
(32, 74)
(48, 74)
(63, 74)
(68, 74)
(70, 74)
(34, 74)
(49, 74)
(44, 74)
(53, 74)
(60, 74)
(73, 74)


Por último, para facilitar futuras aplicaciones, pasemos todos los nombres de las columnas a miníscula

In [81]:
for year in range(0,period):
    df_hitters_free_agents[year].rename(columns = str.lower)
    df_pitchers_free_agents[year].rename(columns = str.lower)
    df_hitters_no_free_agents[year].rename(columns = str.lower)
    df_pitchers_no_free_agents[year].rename(columns = str.lower)

In [82]:
start_year = 2011
end_year = 2022
general_trans_path = 'ETL_Data/Transversal/Cumulative/'

In [83]:
for year in range(0,period):    
    # Exportemos los dataframes por separado
    df_hitters_free_agents[year].to_csv(general_trans_path + '/Hitters/Free_Agent/hitters_' + str(start_year + year) + '.csv',
                                        index = False)
    df_pitchers_free_agents[year].to_csv(general_trans_path + '/Fielders/Free_Agent/fielders_' + str(start_year + year) + '.csv',
                                         index = False)
    df_hitters_no_free_agents[year].to_csv(general_trans_path + '/Hitters/No_Free_Agent/hitters_' + str(start_year + year) + '.csv',
                                           index = False)
    df_pitchers_no_free_agents[year].to_csv(general_trans_path + '/Fielders/No_Free_Agent/fielders_' + str(start_year + year) + '.csv',
                                            index = False)

### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

In [84]:
for year in range(0,period):
    # Condiciones
    condicion_hitter = [df_hitting_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    condicion_pitcher = [df_pitching_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    
    # Etiquetas
    etiquetas = ['Si']
    
    df_hitting_copy[year]['Agente_libre'] = np.select(condicion_hitter, etiquetas, default = 'No')
    df_pitching_copy[year]['Agente_libre'] = np.select(condicion_pitcher, etiquetas, default = 'No')
    
    df_hitting_copy[year] = df_hitting_copy[year].reindex(sorted(df_hitting_copy[year].columns), axis=1)
    df_pitching_copy[year] = df_pitching_copy[year].reindex(sorted(df_pitching_copy[year].columns), axis=1)

In [85]:
for year in range(0,period):
    df_hitting_copy[year].rename(columns = str.lower)
    df_pitching_copy[year].rename(columns = str.lower)

In [86]:
for year in range(0,period):
    # Exportemos los dataframes
    df_hitting_copy[year].to_csv(general_trans_path + '/Hitters/All/hitters_' + str(start_year + year) + '.csv',
                                        index = False)
    df_pitching_copy[year].to_csv(general_trans_path + '/Fielders/All/hitters_' + str(start_year + year) + '.csv',
                                        index = False)

## Panel Data

Con el objetivo de contar con una base de datos en estructura panel, uniremos las bases de datos

In [87]:
# Inicialización del panel
df_panel_all_hitter = df_hitting_copy[0]
df_panel_all_pitcher = df_pitching_copy[0]

for year in range(1,period):
    # Hitter
    df_panel_all_hitter = pd.concat([df_panel_all_hitter, df_hitting_copy[year]])
    
    # Pitcher
    df_panel_all_pitcher = pd.concat([df_panel_all_pitcher, df_pitching_copy[year]])

Veamos las estadísticas descriptivas de los panel

In [88]:
df_panel_all_hitter[['ln_Sueldo_ajustado']].describe()

,ln_Sueldo_ajustado
count,7216.000000
mean,13.779109
std,1.649882
min,8.028455
25%,12.692701
50%,13.226723
75%,15.189226
max,17.440000


In [89]:
df_panel_all_pitcher.describe()

,Altura,Anio_de_agente_libre,Anios_de_contrato,Antiguedad,Bateos,Bateos_2,Bono_por_firma,Cantidad de equipos,Cantidad_agentes_libres,Carreras,...,WHIP,WHIP_2,WS ganadas,Walks,Walks_2,Wins,Wins_2,ln_Sueldo_ajustado,ln_Sueldo_base,ln_Sueldo_regular
count,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000,6.021000e+03,6021.000000,6021.000000,6021.000000,...,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000,6021.000000
mean,6.257905,843.087361,1.471849,0.722637,62.560040,7013.836240,7.734083e+04,2.125727,5.332503,31.831922,...,1.399450,2.444852,3.855838,22.499419,837.511211,3.806344,31.838233,13.502931,13.997466,14.012041
std,0.211507,996.339635,1.206466,0.954166,55.682965,10946.906759,4.995385e+05,1.372477,3.008082,27.776594,...,0.697475,8.707338,5.302430,18.202813,1250.556954,4.165676,61.482440,1.527574,1.149257,1.166916
min,5.567014,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000e+00,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.927685,9.835744,9.546813
25%,6.100000,0.000000,1.000000,0.000000,21.000000,441.000000,0.000000e+00,1.000000,3.000000,11.000000,...,1.150000,1.322500,1.000000,9.000000,81.000000,1.000000,1.000000,12.460362,13.180632,13.182517
50%,6.300000,0.000000,1.000000,1.000000,46.000000,2116.000000,0.000000e+00,2.000000,5.000000,23.000000,...,1.310000,1.716100,2.000000,18.000000,324.000000,2.000000,4.000000,13.161584,13.262125,13.262125
75%,6.400000,2020.000000,1.000000,1.000000,83.000000,6889.000000,0.000000e+00,2.000000,7.000000,44.000000,...,1.500000,2.250000,5.000000,32.000000,1024.000000,5.000000,25.000000,14.626441,14.827111,14.871563
max,6.900000,2029.000000,9.000000,8.000000,314.000000,98596.000000,1.000000e+07,5.000000,18.000000,162.000000,...,21.000000,441.000000,27.000000,106.000000,11236.000000,22.000000,484.000000,17.584433,17.584433,17.584433


In [90]:
df_panel_all_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7216 entries, 0 to 410
Data columns (total 65 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            7216 non-null   object 
 1   Agente_libre                        7216 non-null   object 
 2   Altura                              7216 non-null   float64
 3   Anio                                7216 non-null   object 
 4   Anio_de_agente_libre                7216 non-null   int64  
 5   Anios_de_contrato                   7216 non-null   int64  
 6   Antiguedad                          7216 non-null   int64  
 7   At_bats                             7216 non-null   int64  
 8   At_bats_2                           7216 non-null   int64  
 9   Bateos                              7216 non-null   int64  
 10  Bateos_2                            7216 non-null   int64  
 11  Bateos_promedio                     7216 non

In [91]:
df_panel_all_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6021 entries, 0 to 557
Data columns (total 70 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Acronimo                 6021 non-null   object 
 1   Agente_libre             6021 non-null   object 
 2   Altura                   6021 non-null   float64
 3   Anio                     6021 non-null   object 
 4   Anio_de_agente_libre     6021 non-null   int64  
 5   Anios_de_contrato        6021 non-null   int64  
 6   Antiguedad               6021 non-null   int64  
 7   Bateos                   6021 non-null   int64  
 8   Bateos_2                 6021 non-null   int64  
 9   Bono_por_firma           6021 non-null   int64  
 10  Cantidad de equipos      6021 non-null   int64  
 11  Cantidad_agentes_libres  6021 non-null   int64  
 12  Carreras                 6021 non-null   int64  
 13  Carreras_2               6021 non-null   int64  
 14  Carreras_ganadas         

Verifquemos que no haya problemas con valores *NaN* o *infinitos*

Valores *NaN*:

In [92]:
"""for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"""

"for name in df_panel_all_hitter.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_hitter[name])):\n            if pd.isna(df_panel_all_hitter[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))"

In [93]:
"""for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"""

"for name in df_panel_all_pitcher.columns:\n    print(name)\n    if type(name) != str:\n        for element in range(0,len(df_panel_all_pitcher[name])):\n            if pd.isna(df_panel_all_pitcher[name].iloc[element]) == True:\n                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))"

Valores *infinitos*

In [94]:
for name in df_panel_all_hitter.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_hitter[name])):
            if math.isinf(df_panel_all_hitter[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_hitter[name].iloc[element]))

Acronimo
Agente_libre
Altura
Anio
Anio_de_agente_libre
Anios_de_contrato
Antiguedad
At_bats
At_bats_2
Bateos
Bateos_2
Bateos_promedio
Bateos_promedio_2
Bono_por_firma
Cantidad de equipos
Cantidad_agentes_libres
Dobles
Dobles_2
Edad
Edad_al_firmar
Equipo
Estado
Ganancias
Home_runs
Home_runs_2
Juegos
Juegos totales
Juegos_iniciados
Juegos_iniciados_2
Jugador
Pago_efectivo
Pennants won
Peso
Playoffs
Porcentaje_On_base_plus_slugging
Porcentaje_On_base_plus_slugging_2
Porcentaje_juegos
Porcentaje_juegos_iniciados
Porcentaje_juegos_iniciados_2
Porcentaje_on_base
Porcentaje_on_base_2
Porcentaje_slugging
Porcentaje_slugging_2
Posicion
Promedio_victorias
Runs_batted_in
Runs_batted_in_2
Sueldo_ajustado
Sueldo_base
Sueldo_porcentual
Sueldo_regular
TVS
TVS_2
Triples
Triples_2
Valor_contrato_promedio
Valor_contrato_total
Valor_del_contrato
Victorias
WAR
WAR_2
WS ganadas
ln_Sueldo_ajustado
ln_Sueldo_base
ln_Sueldo_regular


In [95]:
for name in df_panel_all_pitcher.columns:
    print(name)
    if type(name) != str:
        for element in range(0,len(df_panel_all_pitcher[name])):
            if math.isinf(df_panel_all_pitcher[name].iloc[element]) == True:
                print(str(element) +  '  ' + str(df_panel_all_pitcher[name].iloc[element]))

Acronimo
Agente_libre
Altura
Anio
Anio_de_agente_libre
Anios_de_contrato
Antiguedad
Bateos
Bateos_2
Bono_por_firma
Cantidad de equipos
Cantidad_agentes_libres
Carreras
Carreras_2
Carreras_ganadas
Carreras_ganadas_2
Comando
Comando_2
Control
Control_2
Dominio
Dominio_2
ERA
ERA_2
Edad
Edad_al_firmar
Equipo
Estado
Ganancias
Inning_pitched
Inning_pitched_2
Juegos
Juegos totales
Juegos_iniciados
Jugador
Losses
Losses_2
Pago_efectivo
Pennants won
Peso
Playoffs
Porcentaje_juegos
Posicion
Promedio_victorias
Saves
Saves_2
Strike_outs
Strike_outs_2
Sueldo_ajustado
Sueldo_base
Sueldo_porcentual
Sueldo_regular
TVS
TVS_2
Valor_contrato_promedio
Valor_contrato_total
Valor_del_contrato
Victorias
WAR
WAR_2
WHIP
WHIP_2
WS ganadas
Walks
Walks_2
Wins
Wins_2
ln_Sueldo_ajustado
ln_Sueldo_base
ln_Sueldo_regular


In [96]:
df_panel_all_hitter.sort_index(axis = 1,
                               inplace = True)
df_panel_all_pitcher.sort_index(axis = 1,
                                inplace = True)

Exportemos los paneles

In [97]:
general_panel_path = 'ETL_Data/Panel/Cumulative/Visualizations/'

In [98]:
df_panel_all_hitter.to_csv(general_panel_path + 'hitters_' + '.csv',
                           index = False)
df_panel_all_pitcher.to_csv(general_panel_path + 'fielders_' + '.csv',
                            index = False)

Repetiremos el procedimiento, pero únicamente para quienes son agentes libres

In [99]:
# Inicialización del panel
df_panel_fa_hitter = df_hitters_free_agents[0]
df_panel_fa_pitcher = df_pitchers_free_agents[0]

for year in range(1,period):
    # Hitter
    df_panel_fa_hitter = pd.concat([df_panel_fa_hitter, df_hitters_free_agents[year]])
    
    # Pitcher
    df_panel_fa_pitcher = pd.concat([df_panel_fa_pitcher, df_pitchers_free_agents[year]])

In [100]:
df_panel_fa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 670 entries, 0 to 34
Data columns (total 69 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Acronimo                            670 non-null    object 
 1   Altura                              670 non-null    float64
 2   Anio                                670 non-null    object 
 3   Anio_de_agente_libre                670 non-null    int64  
 4   Anios_de_contrato_x                 670 non-null    int64  
 5   Anios_de_contrato_y                 670 non-null    int64  
 6   Antiguedad                          670 non-null    int64  
 7   At_bats                             670 non-null    int64  
 8   At_bats_2                           670 non-null    int64  
 9   Bateos                              670 non-null    int64  
 10  Bateos_2                            670 non-null    int64  
 11  Bateos_promedio                     670 non-nu

In [101]:
df_panel_fa_pitcher.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 0 to 72
Data columns (total 74 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Acronimo                 594 non-null    object 
 1   Altura                   594 non-null    float64
 2   Anio                     594 non-null    object 
 3   Anio_de_agente_libre     594 non-null    int64  
 4   Anios_de_contrato_x      594 non-null    int64  
 5   Anios_de_contrato_y      594 non-null    int64  
 6   Antiguedad               594 non-null    int64  
 7   Bateos                   594 non-null    int64  
 8   Bateos_2                 594 non-null    int64  
 9   Bono_por_firma           594 non-null    int64  
 10  Cantidad de equipos      594 non-null    int64  
 11  Cantidad_agentes_libres  594 non-null    int64  
 12  Carreras                 594 non-null    int64  
 13  Carreras_2               594 non-null    int64  
 14  Carreras_ganadas         59

In [102]:
df_panel_fa_hitter.sort_index(axis = 1, inplace = True)
df_panel_fa_pitcher.sort_index(axis = 1, inplace = True)

# Variables del Modelo Empírico

In [103]:
empiric_panel_hitter = df_panel_fa_hitter.copy()
empiric_panel_pitcher = df_panel_fa_pitcher.copy()

Veamos algunas estadísticas e información que contienen las bases de datos

In [104]:
print(empiric_panel_hitter.shape)

(670, 69)


In [105]:
print(empiric_panel_pitcher.shape)

(594, 74)


Las posiciones que hay en cada base de datos

In [106]:
empiric_panel_hitter['Posicion'].unique()

array(['DH', '3B', 'SP', 'LF', 'SS', 'RP', '2B', '1B', 'C', 'RF', 'CF',
       'RP/CL', 'OF'], dtype=object)

In [107]:
empiric_panel_pitcher['Posicion'].unique()

array(['SP', 'RP', 'RP/CL', 'RF', '1B', 'C', 'DH', 'LF', 'CF', '2B', '3B',
       'SS'], dtype=object)

Ordenemos las bases de datos  por nombre y año

In [108]:
# Hitter
empiric_panel_hitter = empiric_panel_hitter.sort_values(by = ['Jugador','Anio'], ascending=True)
empiric_panel_hitter.reset_index(drop = True, inplace = True)

# Pitcher
empiric_panel_pitcher = empiric_panel_pitcher.sort_values(by = ['Jugador','Anio'], ascending=True)
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

In [109]:
empiric_panel_hitter[['Jugador','Anio']].head()

,Jugador,Anio
0,A.J. Burnett,2014
1,A.J. Burnett,2015
2,A.J. Ellis,2017
3,A.J. Griffin,2016
4,A.J. Pierzynski,2013


In [110]:
empiric_panel_pitcher[['Jugador','Anio']].head()

,Jugador,Anio
0,A.J. Burnett,2014
1,A.J. Burnett,2015
2,A.J. Griffin,2016
3,Aaron Harang,2012
4,Aaron Harang,2015


In [111]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

Obtengamos los índices de las columnas de interes

In [112]:
hitter_regular_stats = ['At_bats_2', 'At_bats',
                        'Bateos_2', 'Bateos',
                        'Bateos_promedio_2', 'Bateos_promedio',
                        'Dobles_2', 'Dobles',
                        'Home_runs_2', 'Home_runs',
                        'Juegos_iniciados_2', 'Juegos_iniciados', 
                        'Porcentaje_On_base_plus_slugging_2', 'Porcentaje_On_base_plus_slugging',
                        'Porcentaje_on_base_2', 'Porcentaje_on_base',
                        'Porcentaje_slugging_2', 'Porcentaje_slugging',
                        'Runs_batted_in_2', 'Runs_batted_in',
                        'Triples_2', 'Triples',
                        'WAR_2', 'WAR']
hitter_regular_stats = sorted(hitter_regular_stats)

In [113]:
# Hitter
for stat in range(0,len(hitter_regular_stats)):
    # Variables auxiliares
    stat_name = hitter_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo":empiric_panel_hitter.groupby(by = "Acronimo")[stat_name].max().index,
                             max_stat_name: empiric_panel_hitter.groupby(by = "Acronimo")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo":empiric_panel_hitter.groupby(by = "Acronimo")[stat_name].min().index,
                             min_stat_name: empiric_panel_hitter.groupby(by = "Acronimo")[stat_name].min().values})
    empiric_panel_hitter = empiric_panel_hitter.merge(max_stat, on = "Acronimo",
                                                      how = "left")
    empiric_panel_hitter = empiric_panel_hitter.merge(min_stat, on = "Acronimo",
                                                      how = "left")

In [114]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [115]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_H')
end = empiric_panel_hitter.columns.get_loc('WAR_L') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_H,At_bats_2_L,Bateos_H,Bateos_L,Bateos_2_H,Bateos_2_L,Bateos_promedio_H,Bateos_promedio_L,Bateos_promedio_2_H,Bateos_promedio_2_L,...,Runs_batted_in_H,Runs_batted_in_L,Runs_batted_in_2_H,Runs_batted_in_2_L,Triples_H,Triples_L,Triples_2_H,Triples_2_L,WAR_H,WAR_L
0,349281,1,156,0,24336,0,0.284,0.00,0.080656,0.0000,...,94,0,8836,0,4,0,16,0,3.49,0.32
1,283024,81,155,1,24025,1,0.291,0.02,0.084681,0.0004,...,88,0,7744,0,5,0,25,0,3.86,0.37
2,412164,1,184,0,33856,0,0.287,0.00,0.082369,0.0000,...,57,0,3249,0,12,0,144,0,4.22,0.22
3,431649,1,178,0,31684,0,0.500,0.00,0.250000,0.0000,...,86,0,7396,0,5,0,25,0,7.73,0.55
4,431649,1,178,0,31684,0,0.500,0.00,0.250000,0.0000,...,86,0,7396,0,5,0,25,0,7.73,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,178084,1,116,0,13456,0,1.000,0.00,1.000000,0.0000,...,80,0,6400,0,2,0,4,0,2.26,0.02
666,328329,1,171,0,29241,0,0.667,0.00,0.444889,0.0000,...,105,0,11025,0,3,0,9,0,6.86,0.13
667,368449,9,173,0,29929,0,0.333,0.00,0.110889,0.0000,...,105,0,11025,0,5,0,25,0,4.81,0.04
668,374544,1,199,0,39601,0,0.500,0.00,0.250000,0.0000,...,100,0,10000,0,4,0,16,0,5.77,0.05


Repitamos el mismo proceso para los lanzadores

In [116]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

Repitamos el mismo proceso para los filderos

In [117]:
pitcher_regular_stats = ['Bateos_2', 'Bateos',
                        'Carreras_2', 'Carreras',
                        'Carreras_ganadas_2', 'Carreras_ganadas',
                        'Comando_2', 'Comando',
                        'Control_2', 'Control',
                        'Dominio_2', 'Dominio', 
                        'ERA_2', 'ERA',
                        'Inning_pitched_2', 'Inning_pitched',
                        'Losses_2', 'Losses',
                        'Saves_2', 'Saves',
                        'Strike_outs_2', 'Strike_outs',
                        'WAR_2', 'WAR',
                        'WHIP_2', 'WHIP',
                        'Walks_2', 'Walks',
                        'Wins_2', 'Wins']
pitcher_regular_stats = sorted(pitcher_regular_stats)

In [118]:
# Pitcheer
for stat in range(0,len(pitcher_regular_stats)):
    # Variables auxiliares
    stat_name = pitcher_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo":empiric_panel_pitcher.groupby(by = "Acronimo")[stat_name].max().index,
                             max_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo":empiric_panel_pitcher.groupby(by = "Acronimo")[stat_name].min().index,
                             min_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo")[stat_name].min().values})
    empiric_panel_pitcher = empiric_panel_pitcher.merge(max_stat, on = "Acronimo",
                                                        how = "left")
    empiric_panel_pitcher = empiric_panel_pitcher.merge(min_stat, on = "Acronimo",
                                                        how = "left")

In [119]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [120]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_H')
end = empiric_panel_pitcher.columns.get_loc('Wins_L') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_H,Bateos_2_L,Carreras_H,Carreras_L,Carreras_2_H,Carreras_2_L,Carreras_ganadas_H,Carreras_ganadas_L,Carreras_ganadas_2_H,Carreras_ganadas_2_L,...,WHIP_H,WHIP_L,WHIP_2_H,WHIP_2_L,Walks_H,Walks_L,Walks_2_H,Walks_2_L,Wins_H,Wins_L
0,42025,361,122,9,14884,81,109,7,11881,49,...,1.65,1.06,2.7225,1.1236,96,5,9216,25,10,0
1,41209,1600,96,15,9216,225,86,15,7396,225,...,1.51,1.06,2.2801,1.1236,71,10,5041,100,13,0
2,44521,1,114,0,12996,0,106,0,11236,0,...,1.63,0.91,2.6569,0.8281,91,0,8281,0,17,0
3,42025,64,101,1,10201,1,83,1,6889,1,...,1.96,0.75,3.8416,0.5625,85,3,7225,9,16,0
4,42025,361,122,9,14884,81,109,7,11881,49,...,1.65,1.06,2.7225,1.1236,96,5,9216,25,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,42025,49,91,2,8281,4,88,2,7744,4,...,2.44,0.86,5.9536,0.7396,76,6,5776,36,16,0
590,42025,64,101,1,10201,1,83,1,6889,1,...,1.96,0.75,3.8416,0.5625,85,3,7225,9,16,0
591,25921,0,80,0,6400,0,77,0,5929,0,...,1.80,0.00,3.2400,0.0000,52,0,2704,0,13,0
592,55696,9,109,1,11881,1,106,1,11236,1,...,3.00,0.69,9.0000,0.4761,74,0,5476,0,15,0


In [121]:
empiric_panel_hitter.shape

(670, 117)

In [122]:
empiric_panel_pitcher.shape

(594, 134)

Lo que se hará ahora es agregar todos los años (12) para cada agente en la base de datos y rellenar los *NaN* faltantes con 0 ya que representan la ausencia de desempeño en este caso.

In [123]:
# Hitter
empiric_panel_hitter = empiric_panel_hitter["Anio"].drop_duplicates().to_frame().merge(empiric_panel_hitter["Jugador"].drop_duplicates(),
                                                                                         how = "cross").merge(empiric_panel_hitter,
                                                                                                              how = "left")
empiric_panel_hitter.reset_index(drop = True, inplace = True)

# Pitcher
empiric_panel_pitcher = empiric_panel_pitcher["Anio"].drop_duplicates().to_frame().merge(empiric_panel_pitcher["Jugador"].drop_duplicates(),
                                                                                           how = "cross").merge(empiric_panel_pitcher,
                                                                                                                how = "left")
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

Veamos las dimensiones

In [124]:
empiric_panel_hitter.shape

(5050, 117)

In [125]:
empiric_panel_pitcher.shape

(4269, 134)

### Filtrando jugadores con más de dos años de observaciones en nuestro panel

In [126]:
# Hitters:
# count number of observations for each player
counts_hitters = empiric_panel_hitter.groupby('Jugador').size()
# filter players with more than one year of observations
filtered_hitters = counts_hitters[counts_hitters > 2].index.tolist()

# Fielders:
# count number of observations for each player
counts_filders = empiric_panel_pitcher.groupby('Jugador').size()
# filter players with more than one year of observations
filtered_fielders = counts_filders[counts_filders > 2].index.tolist()

In [127]:
# Filter observations according to the previous list
# Hitter
empiric_panel_hitter = empiric_panel_hitter[empiric_panel_hitter['Jugador'].isin(filtered_hitters)]
empiric_panel_hitter.reset_index(drop = True,
                                 inplace = True)

# Pitcher
empiric_panel_pitcher = empiric_panel_pitcher[empiric_panel_pitcher['Jugador'].isin(filtered_fielders)]
empiric_panel_pitcher.reset_index(drop = True,
                                  inplace = True)

In [128]:
empiric_panel_hitter

,Anio,Jugador,Acronimo,Altura,Anio_de_agente_libre,Anios_de_contrato_x,Anios_de_contrato_y,Antiguedad,At_bats,At_bats_2,...,Runs_batted_in_2_H,Runs_batted_in_2_L,Triples_H,Triples_L,Triples_2_H,Triples_2_L,WAR_H,WAR_L,WAR_2_H,WAR_2_L
0,2014,A.J. Burnett,PHI,6.500000,2015.0,1.0,2.0,1.0,60.0,3600.0,...,8836.0,0.0,4.0,0.0,16.0,0.0,3.49,0.32,12.1801,0.1024
1,2014,A.J. Ellis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,A.J. Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,A.J. Pierzynski,STL,6.300000,0.0,1.0,1.0,0.0,338.0,114244.0,...,11025.0,0.0,9.0,0.0,81.0,0.0,5.76,0.03,33.1776,0.0009
4,2014,A.J. Pollock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,2018,Yu Darvish,CHC,6.500000,2024.0,6.0,6.0,0.0,12.0,144.0,...,11025.0,0.0,7.0,0.0,49.0,0.0,5.58,0.23,31.1364,0.0529
5046,2018,Yusmeiro Petit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5047,2018,Zach Duke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5048,2018,Zack Cozart,LAA,5.858591,2021.0,3.0,3.0,0.0,224.0,50176.0,...,11025.0,0.0,5.0,0.0,25.0,0.0,4.81,0.04,23.1361,0.0016


In [129]:
empiric_panel_pitcher

,Anio,Jugador,Acronimo,Altura,Anio_de_agente_libre,Anios_de_contrato_x,Anios_de_contrato_y,Antiguedad,Bateos,Bateos_2,...,WHIP_2_H,WHIP_2_L,Walks_H,Walks_L,Walks_2_H,Walks_2_L,Wins_H,Wins_L,Wins_2_H,Wins_2_L
0,2014,A.J. Burnett,PHI,6.5,2015.0,1.0,2.0,1.0,205.0,42025.0,...,2.7225,1.1236,96.0,5.0,9216.0,25.0,10.0,0.0,100.0,0.0
1,2014,A.J. Griffin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,Aaron Harang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,Adam LaRoche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,Adam Ottavino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,2011,Zach Duke,ARI,6.2,0.0,1.0,1.0,1.0,11.0,121.0,...,3.2400,0.0000,52.0,0.0,2704.0,0.0,13.0,0.0,169.0,0.0
4265,2011,Zach Putnam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4266,2011,Zack Britton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4267,2011,Zack Greinke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Para ser consistentes, las columnas que contienen datos de tipo *string* las imputaremos con la palabra *No* ya que representará que no tenía equipo, ni posición, etc.

In [130]:
empiric_panel_hitter.select_dtypes(include =['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Anio                 5050 non-null   object
 1   Jugador              5050 non-null   object
 2   Acronimo             670 non-null    object
 3   Equipo               670 non-null    object
 4   Equipo_anterior      670 non-null    object
 5   Estado               670 non-null    object
 6   Posicion             670 non-null    object
 7   Status_agente_libre  670 non-null    object
dtypes: object(8)
memory usage: 315.8+ KB


In [182]:
empiric_panel_hitter['Posicion'].value_counts()

SP       1018
LF        665
C         617
RP        530
RF        442
2B        410
1B        355
3B        317
SS        228
DH        228
CF        168
RP/CL      44
OF         28
Name: Posicion, dtype: int64

In [132]:
# Fill the NaN values with the previous non-null value
empiric_panel_hitter['Posicion'] = empiric_panel_hitter['Posicion'].fillna(method = 'ffill')

In [133]:
empiric_panel_hitter[['Acronimo',
                      'Equipo_anterior',
                      'Equipo',
                      'Estado',
                      'Posicion',
                      'Status_agente_libre']] = empiric_panel_hitter[['Acronimo',
                      'Equipo_anterior',
                      'Equipo',
                      'Estado',
                      'Posicion',
                      'Status_agente_libre']].fillna('No')

Veamos si funcionó la imputación

In [134]:
empiric_panel_hitter.select_dtypes(include = ['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Anio                 5050 non-null   object
 1   Jugador              5050 non-null   object
 2   Acronimo             5050 non-null   object
 3   Equipo               5050 non-null   object
 4   Equipo_anterior      5050 non-null   object
 5   Estado               5050 non-null   object
 6   Posicion             5050 non-null   object
 7   Status_agente_libre  5050 non-null   object
dtypes: object(8)
memory usage: 315.8+ KB


Ahora con los lanzadores

In [135]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Anio                 4269 non-null   object
 1   Jugador              4269 non-null   object
 2   Acronimo             594 non-null    object
 3   Equipo               594 non-null    object
 4   Equipo_anterior      594 non-null    object
 5   Estado               594 non-null    object
 6   Posicion             594 non-null    object
 7   Status_agente_libre  594 non-null    object
dtypes: object(8)
memory usage: 266.9+ KB


In [183]:
empiric_panel_pitcher['Posicion'].value_counts()

RP       2146
SP       1625
RP/CL     168
RF        125
2B         55
C          37
1B         31
SS         27
3B         20
CF         12
LF         12
DH         11
Name: Posicion, dtype: int64

In [137]:
# Fill the NaN values with the previous non-null value
empiric_panel_pitcher['Posicion'] = empiric_panel_pitcher['Posicion'].fillna(method = 'ffill')

In [138]:
empiric_panel_pitcher[['Acronimo',
                       'Equipo_anterior',
                       'Equipo',
                       'Estado',
                       'Status_agente_libre']] = \
empiric_panel_pitcher[['Acronimo',
                       'Equipo_anterior',
                       'Equipo',
                       'Estado',
                       'Status_agente_libre']].fillna('No')

In [139]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Anio                 4269 non-null   object
 1   Jugador              4269 non-null   object
 2   Acronimo             4269 non-null   object
 3   Equipo               4269 non-null   object
 4   Equipo_anterior      4269 non-null   object
 5   Estado               4269 non-null   object
 6   Posicion             4269 non-null   object
 7   Status_agente_libre  4269 non-null   object
dtypes: object(8)
memory usage: 266.9+ KB


En ambos casos, la imputación fue un éxito. No importa que en periodos de contratación de haya puesto *No* ya que con esto evitamos problemas con posibles instrumentos a partir de dummies.

Por otro lado, repitamos lo mismo para las columnas numéricas, imputaremos 0 ya que refleja la ausencia de desempeño. Sin embargo, la columna que contiene la estadística más alta de cada medida de desempeño se imputará por separado debido a que causará problemas cuando se imponga la trnasformación para el modelo.

In [140]:
# Add string to every element in a list
# Hitter
hitter_high_stat = [stat + '_H' for stat in hitter_regular_stats]
hitter_low_stat = [stat + '_L' for stat in hitter_regular_stats]
# Fielder
pitcher_high_stat = [stat + '_H' for stat in pitcher_regular_stats]
pitcher_low_stat = [stat + '_L' for stat in pitcher_regular_stats]

In [141]:
empiric_panel_hitter[hitter_high_stat] = empiric_panel_hitter[hitter_high_stat].fillna(method = 'ffill')
empiric_panel_pitcher[pitcher_high_stat] = empiric_panel_pitcher[pitcher_high_stat].fillna(method = 'ffill')

In [142]:
empiric_panel_hitter[hitter_high_stat].head(20)

,At_bats_H,At_bats_2_H,Bateos_H,Bateos_2_H,Bateos_promedio_H,Bateos_promedio_2_H,Dobles_H,Dobles_2_H,Home_runs_H,Home_runs_2_H,...,Porcentaje_on_base_H,Porcentaje_on_base_2_H,Porcentaje_slugging_H,Porcentaje_slugging_2_H,Runs_batted_in_H,Runs_batted_in_2_H,Triples_H,Triples_2_H,WAR_H,WAR_2_H
0,591.0,349281.0,156.0,24336.0,0.284,0.080656,28.0,784.0,46.0,2116.0,...,0.348,0.121104,0.504,0.254016,94.0,8836.0,4.0,16.0,3.49,12.1801
1,591.0,349281.0,156.0,24336.0,0.284,0.080656,28.0,784.0,46.0,2116.0,...,0.348,0.121104,0.504,0.254016,94.0,8836.0,4.0,16.0,3.49,12.1801
2,591.0,349281.0,156.0,24336.0,0.284,0.080656,28.0,784.0,46.0,2116.0,...,0.348,0.121104,0.504,0.254016,94.0,8836.0,4.0,16.0,3.49,12.1801
3,607.0,368449.0,173.0,29929.0,0.285,0.081225,50.0,2500.0,32.0,1024.0,...,0.363,0.131769,0.550,0.302500,105.0,11025.0,9.0,81.0,5.76,33.1776
4,607.0,368449.0,173.0,29929.0,0.285,0.081225,50.0,2500.0,32.0,1024.0,...,0.363,0.131769,0.550,0.302500,105.0,11025.0,9.0,81.0,5.76,33.1776
5,607.0,368449.0,173.0,29929.0,0.285,0.081225,50.0,2500.0,32.0,1024.0,...,0.363,0.131769,0.550,0.302500,105.0,11025.0,9.0,81.0,5.76,33.1776
6,607.0,368449.0,173.0,29929.0,0.285,0.081225,50.0,2500.0,32.0,1024.0,...,0.363,0.131769,0.550,0.302500,105.0,11025.0,9.0,81.0,5.76,33.1776
7,607.0,368449.0,173.0,29929.0,0.285,0.081225,50.0,2500.0,32.0,1024.0,...,0.363,0.131769,0.550,0.302500,105.0,11025.0,9.0,81.0,5.76,33.1776
8,607.0,368449.0,173.0,29929.0,0.285,0.081225,50.0,2500.0,32.0,1024.0,...,0.363,0.131769,0.550,0.302500,105.0,11025.0,9.0,81.0,5.76,33.1776
9,607.0,368449.0,173.0,29929.0,0.285,0.081225,50.0,2500.0,32.0,1024.0,...,0.363,0.131769,0.550,0.302500,105.0,11025.0,9.0,81.0,5.76,33.1776


In [143]:
empiric_panel_hitter.fillna(0, inplace = True)
empiric_panel_pitcher.fillna(0, inplace = True)

Verifiquemos si queda alguna columna con alguna entrada tipo *NaN*:

In [144]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [145]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

Obtengamos el máximo de cada una de las medidas de desempeño, de periodos *t_1*, por equipo que han obtenido a lo largo de todas la temporadas

Para evitar problemas con el tipo de *ID*, creemos una que sea numérica para evitar usar los nombres de los jugadores

In [146]:
# Hitter
empiric_panel_hitter['id'] =  empiric_panel_hitter.groupby(['Jugador']).ngroup()
empiric_panel_hitter.reset_index(drop = True, inplace = True)
# Pitcher
empiric_panel_pitcher['id'] =  empiric_panel_pitcher.groupby(['Jugador']).ngroup()
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

Obtengamos la transformación para obtener la $Y$ del modelo empírico a partir de los rezagos de las raices de dichos salarios

In [147]:
# Función de rezagos de raices
def sqrt_dif(X):
    S = []
    for i in range(0, len(X)-1):
        d = np.sqrt(X[i+1])-np.sqrt(X[i])
        S.append(d)
    try:
        S.append(d)
    except: 
        S.append(0)
    return S

In [148]:
Y_hitter = []
for p in empiric_panel_hitter["id"].unique():
    # Filtremos todos los sueldos (ln) de cada jugador por separado
    X = empiric_panel_hitter[empiric_panel_hitter["id"] == p]["ln_Sueldo_ajustado"].values
    # Aplicación de la función
    S = sqrt_dif(X)
    # Añadimos los datos de manera ordenada
    Y_hitter = np.concatenate((Y_hitter, S))
# Agregamos la columna:
empiric_panel_hitter["Y"] = Y_hitter

In [149]:
empiric_panel_hitter["Y"]

0      -0.034934
1      -3.994443
2       0.000000
3       0.000000
4       0.000000
          ...   
5045    0.000000
5046    0.000000
5047    0.000000
5048    0.000000
5049    0.000000
Name: Y, Length: 5050, dtype: float64

In [150]:
Y_pitcher = []
for p in empiric_panel_pitcher["id"].unique():
    # Filtremos todos los sueldos (ln) de cada jugador por separado
    X = empiric_panel_pitcher[empiric_panel_pitcher["id"] == p]["ln_Sueldo_ajustado"].values
    # Aplicación de la función
    S = sqrt_dif(X)
    # Añadimos los datos de manera ordenada
    Y_pitcher = np.concatenate((Y_pitcher, S))
# Agregamos la columna:
empiric_panel_pitcher["Y"] = Y_pitcher

In [151]:
empiric_panel_pitcher["Y"]

0      -0.034934
1      -3.994443
2       0.000000
3       0.000000
4       0.000000
          ...   
4264    0.000000
4265    0.000000
4266    0.000000
4267    0.000000
4268    0.000000
Name: Y, Length: 4269, dtype: float64

Contruyamos las dummy *I* del modelo empírico

In [152]:
# Long of the stats
end_hitter_name = int((len(hitter_high_stat) + len(hitter_low_stat))/2)

In [153]:
hitter_regular_stats

['At_bats',
 'At_bats_2',
 'Bateos',
 'Bateos_2',
 'Bateos_promedio',
 'Bateos_promedio_2',
 'Dobles',
 'Dobles_2',
 'Home_runs',
 'Home_runs_2',
 'Juegos_iniciados',
 'Juegos_iniciados_2',
 'Porcentaje_On_base_plus_slugging',
 'Porcentaje_On_base_plus_slugging_2',
 'Porcentaje_on_base',
 'Porcentaje_on_base_2',
 'Porcentaje_slugging',
 'Porcentaje_slugging_2',
 'Runs_batted_in',
 'Runs_batted_in_2',
 'Triples',
 'Triples_2',
 'WAR',
 'WAR_2']

In [154]:
# Hitter
for sport_stat in range(0,end_hitter_name):
    I_hitter = []
    for y,max_stat,min_stat in zip(empiric_panel_hitter[hitter_regular_stats[sport_stat]],
                                   empiric_panel_hitter[hitter_high_stat[sport_stat]],
                                   empiric_panel_hitter[hitter_low_stat[sport_stat]]):
        # Dummy condition
        if y > (max_stat + min_stat)/2:
            I_hitter.append(0)
        else: 
            I_hitter.append(1)
    
    I_name = hitter_regular_stats[sport_stat] + '_I'
    empiric_panel_hitter[I_name] = I_hitter

Veamos los resultados

In [155]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [156]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [157]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_I')
end = empiric_panel_hitter.columns.get_loc('WAR_I') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_I,Bateos_I,Bateos_2_I,Bateos_promedio_I,Bateos_promedio_2_I,Dobles_I,Dobles_2_I,Home_runs_I,Home_runs_2_I,Juegos_iniciados_I,...,Porcentaje_On_base_plus_slugging_2_I,Porcentaje_on_base_I,Porcentaje_on_base_2_I,Porcentaje_slugging_I,Porcentaje_slugging_2_I,Runs_batted_in_I,Runs_batted_in_2_I,Triples_I,Triples_2_I,WAR_I
0,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,0,0,1,1,1,1,0,...,1,0,0,0,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5045,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5046,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5047,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5048,1,1,1,0,1,1,1,1,1,1,...,0,0,0,0,1,1,1,1,1,1


Repitamos el mismo proceso para los lanzadores

In [158]:
# Long of stats
end_pitcher_name = int((len(pitcher_high_stat) + len(pitcher_low_stat))/2)

In [159]:
# Pitcher
for sport_stat in range(0,end_pitcher_name):
    I_pitcher = []
    for y,max_stat,min_stat in zip(empiric_panel_pitcher[pitcher_regular_stats[sport_stat]],
                                   empiric_panel_pitcher[pitcher_high_stat[sport_stat]],
                                   empiric_panel_pitcher[pitcher_low_stat[sport_stat]]):
        if y > (max_stat + min_stat)/2:
            I_pitcher.append(0)
        else: 
            I_pitcher.append(1)
    
    I_name = pitcher_regular_stats[sport_stat] + '_I'
    empiric_panel_pitcher[I_name] = I_pitcher

In [160]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [161]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

In [162]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_I')
end = empiric_panel_pitcher.columns.get_loc('Wins_I') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_I,Carreras_I,Carreras_2_I,Carreras_ganadas_I,Carreras_ganadas_2_I,Comando_I,Comando_2_I,Control_I,Control_2_I,Dominio_I,...,Saves_2_I,Strike_outs_I,Strike_outs_2_I,WAR_I,WAR_2_I,WHIP_I,WHIP_2_I,Walks_I,Walks_2_I,Wins_I
0,0,0,0,0,0,1,1,0,1,1,...,1,0,0,1,1,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,0,1,1,1,1
4265,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4266,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4267,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


Obtengamos las variables auxiliares

In [163]:
# Good practice
hitter_high_stat = sorted(hitter_high_stat)
# Dummy
I_hitter = sorted([stat + '_I' for stat in hitter_regular_stats])

Verifiquemos que tengan la misma longitud

In [164]:
len(I_hitter) == len(hitter_high_stat) == len(hitter_regular_stats)

True

In [165]:
# Pitcher
for stat in range(0,len(hitter_regular_stats)):
    # Variable auxiliar
    X_hitter = []
    
    # Variables 
    i = (-1)**(empiric_panel_hitter[I_hitter[stat]])
    x = empiric_panel_hitter[hitter_regular_stats[stat]]/np.sqrt(empiric_panel_hitter[hitter_high_stat[stat]])
    X_hitter = i*x
    
    # X name
    name = 'X_' + hitter_regular_stats[stat]
    empiric_panel_hitter[name] = X_hitter

In [166]:
# Pitcher
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

Name: X_Triples
Name: X_Triples_2


In [167]:
# define a list of suffixes to drop
drop_suffix = ['_I', '_L', '_H']

# Use a list comprehension to filter the columns to drop
hitter_cols_to_drop = [col for col in empiric_panel_hitter.columns if col.endswith(tuple(drop_suffix))]

# Drop the selected columns
empiric_panel_hitter = empiric_panel_hitter.drop(hitter_cols_to_drop,
                                                 axis=1)

In [168]:
empiric_panel_hitter.shape

(5050, 95)

In [169]:
empiric_panel_hitter['Posicion'].value_counts()

SP       1018
LF        665
C         617
RP        530
RF        442
2B        410
1B        355
3B        317
SS        228
DH        228
CF        168
RP/CL      44
OF         28
Name: Posicion, dtype: int64

In [170]:
general_dynamic_path = 'ETL_Data/Panel/Cumulative/Dynamic_model/'

In [171]:
empiric_panel_hitter.to_csv(general_dynamic_path + 'panel_hitters' + '.csv',
                            index = False)

Ahora, filtremos las columnas que contienen las medidas de desempeño de los periodos **t_1**. Para ello, encontremos primero los índices de dichas variables

In [172]:
# Pitcher
for stat in range(0,len(hitter_regular_stats)):
    # Stat
    print(colored(hitter_regular_stats[stat], "cyan"))
    
    # X name
    name = 'X_' + hitter_regular_stats[stat]
    
    # OLS variables
    Y = empiric_panel_hitter['Y'].tolist()
    X = empiric_panel_hitter[name].tolist()
    X = sm.add_constant(X)
    
    # Modelo
    model = sm.OLS(Y, X,
                   missing = 'drop').fit()
    print(model.summary())
    print("\n")

At_bats
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.4580
Date:                Mon, 13 Mar 2023   Prob (F-statistic):              0.499
Time:                        12:48:25   Log-Likelihood:                -10235.
No. Observations:                5050   AIC:                         2.047e+04
Df Residuals:                    5048   BIC:                         2.049e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0045      0.026     -0.175 

Repitamos el mismo procedimiento para los filderos

In [173]:
# Good practice
pitcher_high_stat = sorted(pitcher_high_stat)
# Dummy
I_pitcher = sorted([stat + '_I' for stat in pitcher_regular_stats])

Verifiquemos que tengan la misma longitud

In [174]:
len(I_pitcher) == len(pitcher_high_stat) == len(pitcher_regular_stats)

True

In [175]:
# Pitcher
for stat in range(0,len(pitcher_regular_stats)):
    # Variable auxiliar
    X_pitcher = []
    
    # Variables 
    i = (-1)**(empiric_panel_pitcher[I_pitcher[stat]])
    x = empiric_panel_pitcher[pitcher_regular_stats[stat]]/np.sqrt(empiric_panel_pitcher[pitcher_high_stat[stat]])
    X_pitcher = i*x
    
    # X name
    name = 'X_' + pitcher_regular_stats[stat]
    empiric_panel_pitcher[name] = X_pitcher

In [176]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

Tenemos que borrar las columnas que terminan con '_L', '_H' o '_I'

In [177]:
# define a list of suffixes to drop
drop_suffix = ['_I', '_L', '_H']

# Use a list comprehension to filter the columns to drop
fielder_cols_to_drop = [col for col in empiric_panel_pitcher.columns if col.endswith(tuple(drop_suffix))]

# Drop the selected columns
empiric_panel_pitcher = empiric_panel_pitcher.drop(fielder_cols_to_drop,
                                                   axis=1)

In [178]:
empiric_panel_pitcher.shape

(4269, 106)

In [179]:
empiric_panel_pitcher['Posicion'].value_counts()

RP       2146
SP       1625
RP/CL     168
RF        125
2B         55
C          37
1B         31
SS         27
3B         20
CF         12
LF         12
DH         11
Name: Posicion, dtype: int64

In [180]:
empiric_panel_pitcher.to_csv(general_dynamic_path + 'panel_fielders' + '.csv',
                             index = False)

Ahora, filtremos las columnas que contienen las medidas de desempeño de los periodos **t_1**. Para ello, encontremos primero los índices de dichas variables

In [181]:
# Pitcher
for stat in range(0,len(pitcher_regular_stats)):
    # Stat
    print(colored(pitcher_regular_stats[stat], "cyan"))
    
    # X name
    name = 'X_' + pitcher_regular_stats[stat]
    
    # OLS variables
    Y = empiric_panel_pitcher['Y'].tolist()
    X = empiric_panel_pitcher[name].tolist()
    X = sm.add_constant(X)
    
    # Modelo
    model = sm.OLS(Y, X,
                   missing = 'drop').fit()
    print(model.summary())
    print("\n")

Bateos
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                   0.05166
Date:                Mon, 13 Mar 2023   Prob (F-statistic):              0.820
Time:                        12:48:26   Log-Likelihood:                -8628.6
No. Observations:                4269   AIC:                         1.726e+04
Df Residuals:                    4267   BIC:                         1.727e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0733      0.028     -2.619  